Daisy Johanna Uy

Final Model (Business & Analytics Program)

## Import required packages and set global settings

In [ ]:
%matplotlib inline

from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns
!pip install scipy #category_encoders dependent on these packages
!pip install scikit-plot
!pip install scikit-learn
!pip install category_encoders
import category_encoders as ce
import matplotlib.pylab as plt
!pip install dmba
from dmba import plotDecisionTree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 11.6 MB/s 
no display found. Using non-interactive Agg backend


## Importing and initial cleaning of the data

In [ ]:
wfh_raw = pd.read_csv('WFHData_July21toJune22.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (371,381) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#removing any spaces
wfh_raw.columns = wfh_raw.columns.str.strip()

# Dropping features that are unimportant based on EDA

In [ ]:
wfh2 = wfh[['date', 'logpop_den_job_current', 'workhours_duringCOVID', 'wfhcovid_ever', 'work_computer_pct', 'workhours_preCOVID', 'wfh_eff_COVID_quant', 'hourly_wage',
            'commutetime_quant', 'drivealone_current_pct', 'nocommute_current_pct', 'employer_sizecat', 'uploadspeed', 'downloadspeed', 'work_industry', 'worktime_nonremoteable_pct',
            'worktime_remoteable_pct', 'occupation_clean', 'extratime_1stjob', 'wfh_interviewing', 'groomtime_commute', 'self_employment', 'extratime_2ndjob', 'extratime_indoorleisure',
            'workstatus_current']] #24 features

#Exploring our chosen features

In [ ]:
#changing the date object to datetime format
wfh2['date'] = pd.to_datetime(wfh2['date'], format='%Ym%m')
wfh2['quit_date'] = pd.to_datetime(wfh2['quit_date'], format='%Ym%m')

In [ ]:
wfh2['day'] = wfh2['date'].dt.day
wfh2['month'] = wfh2['date'].dt.month
wfh2['year'] = wfh2['date'].dt.year

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
wfh2 = wfh2.drop(labels=['date'], axis=1)

In [ ]:
list(wfh2.columns.values)

['logpop_den_job_current',
 'workhours_duringCOVID',
 'wfhcovid_ever',
 'work_computer_pct',
 'workhours_preCOVID',
 'wfh_eff_COVID_quant',
 'hourly_wage',
 'commutetime_quant',
 'drivealone_current_pct',
 'nocommute_current_pct',
 'employer_sizecat',
 'uploadspeed',
 'downloadspeed',
 'work_industry',
 'worktime_nonremoteable_pct',
 'worktime_remoteable_pct',
 'occupation_clean',
 'extratime_1stjob',
 'wfh_interviewing',
 'groomtime_commute',
 'self_employment',
 'extratime_2ndjob',
 'extratime_indoorleisure',
 'workstatus_current',
 'day',
 'month',
 'year']

In [ ]:
wfh2 = wfh2[['year', 'month', 'day', 'logpop_den_job_current',
 'workhours_duringCOVID',
 'wfhcovid_ever',
 'work_computer_pct',
 'workhours_preCOVID',
 'wfh_eff_COVID_quant',
 'hourly_wage',
 'commutetime_quant',
 'drivealone_current_pct',
 'nocommute_current_pct',
 'employer_sizecat',
 'uploadspeed',
 'downloadspeed',
 'work_industry',
 'worktime_nonremoteable_pct',
 'worktime_remoteable_pct',
 'occupation_clean',
 'extratime_1stjob',
 'wfh_interviewing',
 'groomtime_commute',
 'self_employment',
 'extratime_2ndjob',
 'extratime_indoorleisure',
 'workstatus_current']]

# Splitting the dataset

In [ ]:
X = wfh2.drop(columns=['workstatus_current'])
y = wfh2['workstatus_current']

train_X, valid_X, train_y, valid_y = train_test_split(X,y, test_size=0.3, random_state=1)

# Impute missing values in training data

In [ ]:
X_imp = imp.fit(train_X2)

In [ ]:
X_imp_array = imp.transform(train_X2)

In [ ]:
train_X.columns

Index(['year', 'month', 'day', 'logpop_den_job_current',
       'workhours_duringCOVID', 'wfhcovid_ever', 'work_computer_pct',
       'workhours_preCOVID', 'wfh_eff_COVID_quant', 'hourly_wage',
       'commutetime_quant', 'drivealone_current_pct', 'nocommute_current_pct',
       'employer_sizecat', 'uploadspeed', 'downloadspeed', 'work_industry',
       'worktime_nonremoteable_pct', 'worktime_remoteable_pct',
       'occupation_clean', 'extratime_1stjob', 'wfh_interviewing',
       'groomtime_commute', 'self_employment', 'extratime_2ndjob',
       'extratime_indoorleisure'],
      dtype='object')

In [ ]:
train_Xi = pd.DataFrame(X_imp_array,
                       columns=['year', 'month', 'day', 'logpop_den_job_current',
       'workhours_duringCOVID', 'wfhcovid_ever', 'work_computer_pct',
       'workhours_preCOVID', 'wfh_eff_COVID_quant', 'hourly_wage',
       'commutetime_quant', 'drivealone_current_pct', 'nocommute_current_pct',
       'employer_sizecat', 'uploadspeed', 'downloadspeed', 'work_industry',
       'worktime_nonremoteable_pct', 'worktime_remoteable_pct',
       'occupation_clean', 'extratime_1stjob', 'wfh_interviewing',
       'groomtime_commute', 'self_employment', 'extratime_2ndjob',
       'extratime_indoorleisure'])

#Impute test data

In [ ]:
train_Xi.mode()

,year,month,day,logpop_den_job_current,workhours_duringCOVID,wfhcovid_ever,work_computer_pct,workhours_preCOVID,wfh_eff_COVID_quant,hourly_wage,commutetime_quant,drivealone_current_pct,nocommute_current_pct,employer_sizecat,uploadspeed,downloadspeed,work_industry,worktime_nonremoteable_pct,worktime_remoteable_pct,occupation_clean,extratime_1stjob,wfh_interviewing,groomtime_commute,self_employment,extratime_2ndjob,extratime_indoorleisure
0,2022.0,6.0,1.0,10.364245,40.0,100.0,100.0,40.0,0.0,17.5,30.0,100.0,0.0,5.0,100.0,100.0,6.0,100.0,100.0,8.0,0.0,1.0,30.0,1.0,0.0,10.0


In [ ]:
values = {'workhours_duringCOVID': 40, 'work_computer_pct': 100, 'workhours_preCOVID': 40, 'logpop_den_job_current': 10.364245,
           'wfh_eff_COVID_quant' : 0, 'worktime_nonremoteable_pct' : 100, 'worktime_remoteable_pct' : 100, 'drivealone_current_pct' : 100, 'nocommute_current_pct' : 0,
           'extratime_1stjob' : 0, 'employer_sizecat' : 5, 'uploadspeed' : 100, 'downloadspeed' : 100, 'hourly_wage' : 17.5, 'occupation_clean' : 8, 'work_industry' : 6,
           'wfh_interviewing': 1, 'groomtime_commute' : 30, 'self_employment' : 1, 'extratime_2ndjob' : 0, 'extratime_indoorleisure' : 10}

In [ ]:
valid_X = valid_X.fillna(value = values)

In [ ]:
valid_X.head()

,year,month,day,logpop_den_job_current,workhours_duringCOVID,wfhcovid_ever,work_computer_pct,workhours_preCOVID,wfh_eff_COVID_quant,hourly_wage,...,work_industry,worktime_nonremoteable_pct,worktime_remoteable_pct,occupation_clean,extratime_1stjob,wfh_interviewing,groomtime_commute,self_employment,extratime_2ndjob,extratime_indoorleisure
5669,2021,9,1,11.297915,20.0,100,100.0,20,12.5,225.000000,...,4.0,100.0,100.0,2.0,10.0,1.0,30.0,1.0,20.0,20.0
31973,2021,10,1,6.161024,37.0,0,100.0,50,0.0,6.000000,...,6.0,100.0,100.0,10.0,0.0,1.0,30.0,1.0,0.0,10.0
8249,2021,7,1,8.174906,19.0,100,100.0,20,0.0,1000.000000,...,2.0,100.0,100.0,10.0,10.0,1.0,30.0,1.0,10.0,10.0
33521,2021,11,1,7.810554,70.0,0,100.0,60,0.0,18.055555,...,17.0,100.0,0.0,10.0,0.0,1.0,20.0,2.0,0.0,10.0
15006,2022,1,1,6.038852,40.0,100,100.0,40,-20.0,22.500000,...,9.0,100.0,75.0,6.0,0.0,1.0,15.0,1.0,0.0,50.0


# Random Forest Classifier on wfh1 and wfh2

In [ ]:
rf = RandomForestClassifier(n_estimators=500, random_state=1)

In [ ]:
rf.fit(train_Xi, train_y)

RandomForestClassifier(n_estimators=500, random_state=1)

In [ ]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

In [ ]:
rf_1 = pd.DataFrame({'feature': train_Xi.columns, 'importance': importances, 'std': std})
rf_1 = rf_1.sort_values('importance')
print(rf_1)

                       feature  importance       std
2                          day    0.000000  0.000000
0                         year    0.005246  0.001647
12       nocommute_current_pct    0.013089  0.008381
17  worktime_nonremoteable_pct    0.015221  0.005037
6            work_computer_pct    0.021484  0.006795
18     worktime_remoteable_pct    0.025612  0.008610
19            occupation_clean    0.026264  0.002812
1                        month    0.027081  0.004048
11      drivealone_current_pct    0.030892  0.013315
16               work_industry    0.030896  0.002115
13            employer_sizecat    0.034346  0.014609
14                 uploadspeed    0.037007  0.002155
15               downloadspeed    0.038049  0.002365
10           commutetime_quant    0.052709  0.017991
9                  hourly_wage    0.055734  0.015613
7           workhours_preCOVID    0.055838  0.027336
8          wfh_eff_COVID_quant    0.057470  0.029265
20            extratime_1stjob    0.059618  0.

In [ ]:
y_pred = rf.predict(valid_X)

In [ ]:
# checking accuracy for validation set
result = metrics.multilabel_confusion_matrix(valid_y, y_pred)
print("Multi-label Confusion Matrix:")
print(result)

result2 = metrics.classification_report(valid_y, y_pred)
print("Classification Report:")
print(result2)

result3 = metrics.accuracy_score(valid_y, y_pred)
print("Accuracy:",result3)

print('Mean Absolute Error:', metrics.mean_absolute_error(valid_y, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(valid_y, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(valid_y, y_pred)))

Confusion Matrix:
[[4883 1997   97]
 [ 921 5185   89]
 [  34   19 3594]]
Multi-label Confusion Matrix:
[[[ 8887   955]
  [ 2094  4883]]

 [[ 8608  2016]
  [ 1010  5185]]

 [[12986   186]
  [   53  3594]]]
Classification Report:
              precision    recall  f1-score   support

           1       0.84      0.70      0.76      6977
           2       0.72      0.84      0.77      6195
           3       0.95      0.99      0.97      3647

    accuracy                           0.81     16819
   macro avg       0.84      0.84      0.83     16819
weighted avg       0.82      0.81      0.81     16819

Accuracy: 0.8122956180510137
Mean Absolute Error: 0.19549319222308104
Mean Squared Error: 0.2110708127712706
Root Mean Squared Error: 0.4594244364106796
R-squared Score: 0.643742127355236
